In [1]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import csv
import fasttext
import re 
from sklearn.model_selection import train_test_split


In [3]:
df = pd.read_csv('/content/dataset.csv')

In [4]:
def tokenization(query):
    tokens=query.upper()
    if "WHERE" in tokens:
        tokens=tokens.split("WHERE",1)
        tokens=tokens[1]
    Dict = { '!=': 'NEQ', '<>': 'NEQ', '&&': 'AND', '||': 'OR', '/*': ' CMTST ',
            '*/': 'CMTEND',
            '~': 'TLDE', '!': 'EXCLM', '@': 'ATR', '#': 'HASH', '$': 'DLLR', '%': 'PRCNT',
            '^': 'XOR', '&': 'BITAND','|': 'BITOR', '*': 'STAR', '--': 'CMNT', '-': 'MINUS',
            '+': 'PLUS', '=': 'EQ', '/': 'SLSH', '?': 'QSTN','.': 'DOT',
            ',': 'CMMA', '>': 'GT', '<': 'LT', '\'': 'SQUT', '"': 'DQUT', ';': 'SMCLN', 
            ':': 'CLN', '\\': 'BSLSH',
            ']': 'RSQBR', '[': 'LSQBR', '}': 'RCBR', '{': 'LCBR', ')': 'RPRN', '(': 'LPRN'}
    i=0 #counter
    while i < len(tokens):
        if tokens[i] in Dict: #if token is in dict
            if i + 1 < len(tokens) and (tokens[i] + tokens[i + 1]) in Dict: #if two things after each other replace such as ||
                tokens=tokens.replace((tokens[i] + tokens[i + 1])," "+Dict[(tokens[i] + tokens[i + 1])]+" ")
            else:
                tokens=tokens.replace(tokens[i], " "+Dict[tokens[i]]+" ")# if only 1 replace it
           
        i = i + 1 #increment counter
    tokens =re.sub(r"(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})"," LNK ",tokens)
    tokens =re.sub(r"((?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?\.){3}(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)))"," IPADDR ",tokens)
    tokens =re.sub(r"\d+"," INT ",tokens)
    tokens =re.sub(r"([A-Fa-f0-9]{2,})"," HEX ",tokens)
    tokens =re.sub(r"^\d*[.,]?\d*$"," DEC ",tokens)
    tokens =re.sub(r"\b[a-zA-Z]\b"," CHR ",tokens)
       
    return "".join(tokens) # convert it to string again

In [5]:
df.iloc[:, 1] = df.iloc[:, 1].apply(lambda x: tokenization(str(x))) 

In [6]:
train, test = train_test_split(df, test_size=0.2)

In [7]:
train[['Label', 'Query']].to_csv('train.txt', 
                                          index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ")

test[['Label', 'Query']].to_csv('test.txt', 
                                     index = False, 
                                     sep = ' ',
                                     header = None, 
                                     quoting = csv.QUOTE_NONE, 
                                     quotechar = "", 
                                     escapechar = " ")

In [8]:
model = fasttext.train_supervised('train.txt', lr=0.1)


In [9]:
total, precision, recall = model.test('test.txt') #test the model
Fmeasure= 2 * ((precision*recall)/(precision + recall)) #calc f-measure
print(str(Fmeasure * 100) + "%" )

98.64864864864865%
